<a href="https://colab.research.google.com/github/framante/NAPDE-project-NN/blob/main/conv1D_exchanged_order_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers.core import Reshape

REMEMBER THAT IN PYTHON THE FIRST DIMENSION OF A TENSOR IS THE WIDTH, WHICH I USUALLY CONSIDER AS THE THIRD ONE, while the remaining others are the dimension of a classical matrix n rows x n cols

In [ ]:
#training data

# FINAL STEP: here we implement the version for all FEM nodes (hopefully)

# number of FEM nodes (control nodes in Higa setting)
n_FEMnodes = 11
# number of parameters(mu, beta1, beta2, sigma, f)
n_params = 5
# number of PDEs problems considered
n_PDEs = 200
# number of quadrature nodes for each FEM node
data_size = 100 #(expected_output.shape[1])

#import expected output dataset
expected_output = np.loadtxt('SolutionOutput.txt')
print(expected_output.shape) # n FEM nodes x n PDEs x n QUAD nodes

# import input dataset (think about importing n PDEs files instead SolutionInput_1.txt ... SolutionInput_nPDEs.txt)
input = np.loadtxt('SolutionInput.txt')
print(input.shape) # n FEM nodes x n PDEs x n params x n QUAD nodes

(2200, 100)
(11000, 100)


Reshaping the expected output such that it's a tensor where each matrix is (n FEM nodes x n QUAD nodes) and the third dimension is the n PDEs

In [ ]:
expected_output_rshpd = expected_output.reshape([n_PDEs, n_FEMnodes, data_size])
print(expected_output_rshpd.shape) # n PDEs x n FEM nodes x n QUAD nodes

(200, 11, 100)


"Rolling the dice" so that we have a tensor where each matrix is (n PDEs x n QUAD nodes) and the third dimension is n FEM nodes

In [ ]:
#expected_output_rshpd = np.swapaxes(expected_output_rshpd, 0, 1)
#print(expected_output_rshpd.shape) # n FEM nodes x n PDEs x n QUAD nodes

Reshaping input

In [ ]:
input_rshpd = input.reshape([n_PDEs, n_FEMnodes, n_params, data_size])
print(input_rshpd.shape) # n PDEs x n FEM nodes x n params x n QUAD nodes

(200, 11, 5, 100)


In [ ]:
#input_rshpd = np.swapaxes(input_rshpd, 0, 1)
#print(input_rshpd.shape) # n FEM nodes x n PDEs x n params x n QUAD nodes

The training data should be interpreted as input and expected output


the input is given by the terms (mu, beta1, beta2, sigma, force) of the ADR problem  computed in the grid (n FEM nodes x n QUAD nodes)

the output is given by the kind-of-solution (...specify how it has been got....) evaluated in the grid (n FEM nodes x n QUAD nodes)

In [ ]:
x_train = input_rshpd
#x_train = np.expand_dims(x_train, 4) 
y_train = expected_output_rshpd

print(x_train.shape)
print(y_train.shape) 

(200, 11, 5, 100)
(200, 11, 100)


training for all n_FEMnodes, the tuning of the width of the window is done within the layers conv1D

In [ ]:
input_shape = (n_FEMnodes, n_params, data_size)
print(input_shape)

model = tf.keras.Sequential()

layer1 = tf.keras.Input(shape=input_shape)
model.add(layer1)

layer2 = tf.keras.layers.Dense(40, activation="relu")
model.add(layer2)

layer3 = tf.keras.layers.Conv1D(20, kernel_size=(5,), activation="relu")
model.add(layer3)

layer4 = tf.keras.layers.Dropout(0.2)
model.add(layer4)

#layer5 = tf.keras.layers.Conv1D(30, kernel_size=2, activation="relu")
#model.add(layer5)

#layer6 = tf.keras.layers.Dropout(0.2)
#model.add(layer6)

# layer = tf.keras.layers.MaxPool1D(pool_size=2)
# we cannot use a pooling layer since it requires the dimension of input to be : (batch_shape, steps, features)
# and we have instead 4 dimensions

#layer8 = tf.keras.layers.Conv1D(10, kernel_size=2, activation="relu")
#model.add(layer8)

model.add(tf.keras.layers.Reshape((n_FEMnodes, 20)) )

layer9 = tf.keras.layers.Conv1D(10, kernel_size=3, padding="same", activation="relu")
model.add(layer9)

layer10 = tf.keras.layers.Dropout(0.5)
model.add(layer10)

layer11 = tf.keras.layers.Dense(data_size)
model.add(layer11)

#tf.keras.layers.MaxPool1D(pool_size=2)
#tf.keras.layers.Flatten()

#layer11 = tf.keras.layers.Dense(data_size)
#model.add(layer11)

(11, 5, 100)


In [ ]:
print(input_shape)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, "model_shape_info.png", show_shapes=True)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01,rho=0.9), 
              loss = "mse"
              #,metrics=tf.keras.metrics.MeanSquaredLogarithmicError( name='msl_err', dtype=None)
              )

In [ ]:
batch_size = 20
epochs = 200
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split = 0.2)

Epoch 1/200
8/8 [==============================] - 1s 33ms/step - loss: 1.3712 - val_loss: 0.0056
Epoch 2/200
8/8 [==============================] - 0s 11ms/step - loss: 0.0110 - val_loss: 0.0042
Epoch 3/200
8/8 [==============================] - 0s 11ms/step - loss: 0.0070 - val_loss: 0.0036
Epoch 4/200
8/8 [==============================] - 0s 11ms/step - loss: 0.0106 - val_loss: 0.0033
Epoch 5/200
8/8 [==============================] - 0s 10ms/step - loss: 0.0071 - val_loss: 0.0033
Epoch 6/200
8/8 [==============================] - 0s 10ms/step - loss: 0.0070 - val_loss: 0.0033
Epoch 7/200
8/8 [==============================] - 0s 11ms/step - loss: 0.0079 - val_loss: 0.0033
Epoch 8/200
8/8 [==============================] - 0s 12ms/step - loss: 0.0085 - val_loss: 0.0033
Epoch 9/200
8/8 [==============================] - 0s 10ms/step - loss: 0.0055 - val_loss: 0.0035
Epoch 10/200
8/8 [==============================] - 0s 11ms/step - loss: 0.0063 - val_loss: 0.0035
Epoch 11/200
8/8 [=

# **Extract testing data the same way**







In [ ]:
#testing data

# number of FEM nodes (control nodes in Higa setting)
n_FEMnodes = 11
# number of parameters(mu, beta1, beta2, sigma, f)
n_params = 5
# number of PDEs problems considered (CHANGED WRT TRAINING STAGE OF COURSE)
t_PDEs = 1
# number of quadrature nodes for each FEM node
data_size = 100 #(expected_output.shape[1])

#import expected output dataset
t_expected_output = np.loadtxt('TestOutput.txt')
print(t_expected_output.shape) # n FEM nodes x t PDEs x n QUAD nodes

# import input dataset (think about importing n PDEs files instead SolutionInput_1.txt ... SolutionInput_nPDEs.txt)
t_input = np.loadtxt('TestInput.txt')
print(t_input.shape) # n FEM nodes x n PDEs x t params x n QUAD nodes

(11, 100)
(55, 100)


Reshaping testing expected output

In [ ]:
t_expected_output_rshpd = t_expected_output.reshape([t_PDEs, n_FEMnodes, data_size])
print(t_expected_output_rshpd.shape) # t PDEs x n FEM nodes x n QUAD nodes

(1, 11, 100)


In [ ]:
#t_expected_output_rshpd = np.swapaxes(t_expected_output_rshpd, 0, 1)
#print(t_expected_output_rshpd.shape) # t FEM nodes x n PDEs x n QUAD nodes

Reshaping testing input

In [ ]:
t_input_rshpd = t_input.reshape([t_PDEs, n_FEMnodes, n_params, data_size])
print(t_input_rshpd.shape) # t PDEs x n FEM nodes x n params x n QUAD nodes

(1, 11, 5, 100)


In [ ]:
#t_input_rshpd = np.swapaxes(t_input_rshpd, 0, 1)
#print(t_input_rshpd.shape) # t FEM nodes x n PDEs x n params x n QUAD nodes

Evaluate the model through the testing dataset

In [ ]:
x_test = t_input_rshpd
#x_test = np.expand_dims(x_test, 4) 
y_test = t_expected_output_rshpd

print(x_test.shape)
print(y_test.shape) 

(1, 11, 5, 100)
(1, 11, 100)


In [ ]:
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

1/1 [==============================] - 0s 169ms/step - loss: 0.0037
Test loss: 0.0037447893992066383


Predict the output for the desidered input

In [ ]:
result = model.predict(x_test, batch_size=50)
print(result.shape)

(1, 11, 100)


In [ ]:
result

Write a file containing the results of the output of the CNN on the input x_test

In [ ]:
result = result.reshape((n_FEMnodes, data_size))
f = open("output_ML.txt", "w")

for i in range(result.shape[0]):
  for j in range(result.shape[1]):
    f.write(str(result[i][j]))
    f.write("  ")
  f.write("\n")

f.close()